In [128]:
import numpy as np
import re

In [129]:
def read_file(file_name):
    with open(file_name, 'r') as f:
        return f.read()

In [130]:
file_content = read_file('test.mei')

def find_measure(file_content):
    measures = re.findall(r'<measure[\s\S]*?measure>', file_content)
    return measures

measures = find_measure(file_content)

In [161]:
def extract_measure_content(measure):
    content = re.findall(r'<note.*?/>|<rest.*?/>|<multiRest.*?/>', measure)
    return content

measures_content = {}

octaves = {0:',,,,,', 1:',,,,', 2:',,,', 3:',,', 4:',', 5:'', 6:"'", 7:"''"}
duration_mapping = {16:1, 8:2, 4:4, 2:8, 1:16}

for i, measure in enumerate(measures):
    measure_content = extract_measure_content(measure)
    measure_notes = []
    for j, symbol in enumerate(measure_content):
        print('bbbbbbb', symbol)
        if symbol.startswith('<note'):
            value = re.findall(r'pname="([^"]*)"', symbol)[0]
            octave = octaves[int(re.findall(r'oct="([^"]*)"', symbol)[0])]
            duration = duration_mapping[int(re.findall(r'dur="([^"]*)"', symbol)[0])]
            dot = re.findall(r'dots="([^"]*)"', symbol)
            if dot:
                duration = f'{int(duration*1.5)}'
            note = f'{value}{octave}{duration}'
            measure_notes.append(note)
        
        if symbol.startswith('<rest'):
            duration = duration_mapping[int(re.findall(r'dur="([^"]*)"', symbol)[0])]
            rest = f'z{duration}'
            measure_notes.append(rest)

        if symbol.startswith('<multiRest'):
            duration = re.findall(r'num="([^"]*)"', symbol)[0]
            rest = f'Z{duration}'
            measure_notes.append(rest)
            
    measures_content[i] = measure_notes

measures_content


bbbbbbb <multiRest xml:id="multirest-0000000266443102" num="23" />
bbbbbbb <rest xml:id="rest-0000000866234707" dur="4" />
bbbbbbb <rest xml:id="rest-0000001015077536" dur="8" />
bbbbbbb <note xml:id="note-0000000798055784" dur="8" oct="4" pname="b" />
bbbbbbb <note xml:id="note-0000000071140080" dots="1" dur="4" oct="4" pname="b" />
bbbbbbb <note xml:id="note-0000001650416828" dur="8" oct="4" pname="g" />
bbbbbbb <note xml:id="note-0000000428769533" dots="1" dur="4" oct="5" pname="e" />
bbbbbbb <note xml:id="note-0000001521905446" dur="8" oct="5" pname="d" />
bbbbbbb <note xml:id="note-0000000841346155" dur="8" oct="5" pname="c" />
bbbbbbb <note xml:id="note-0000001472495237" dur="8" oct="5" pname="c" />
bbbbbbb <rest xml:id="rest-0000000625900231" dur="4" />


{0: ['Z23'],
 1: ['z4', 'z2', 'b,2'],
 2: ['b,6', 'g,2'],
 3: ['e6', 'd2'],
 4: ['c2', 'c2', 'z4']}

In [164]:
gammes = {'0':'C', '1s':'G', '2s':'D', '3s':'A', '4s':'E', '5s':'B', '6s':'F#', '7s':'C#', '1f':'F', '2f':'Bb', '3f':'Eb', '4f':'Ab', '5f':'Db', '6f':'Gb', '7f':'Cb'}

def find_score_def(file_content):
    score_def = re.findall(r'<scoreDef[\s\S]*?scoreDef>', file_content)
    key =  re.findall(r'key.sig="([^"]*)"', score_def[0])
    meter_count = re.findall(r'meter.count="([^"]*)"', score_def[0])
    meter_unit = re.findall(r'meter.unit="([^"]*)"', score_def[0])
    staff_def = re.findall(r'<staffDef.*?/>', score_def[0])[0]
    if staff_def:
        clef_shape = re.findall(r'clef.shape="([^"]*)"', staff_def)[0]
        clef_line = re.findall(r'clef.line="([^"]*)"', staff_def)[0]
    else:
        clef_shape = ''
        clef_line = ''

    score_def = {'key': gammes[key[0]], 'meter_count': int(meter_count[0]), 'meter_unit': int(meter_unit[0]), 'clef': clef_shape+clef_line}
    return score_def

score_def = find_score_def(file_content)
print(score_def)

{'key': 'Eb', 'meter_count': 2, 'meter_unit': 4, 'clef': 'C1'}


In [165]:
def mei_to_abc(measures_content, score_def):
    abc_content = ''
    abc_content += f'X:1\n'
    abc_content += f'M:{score_def["meter_count"]}/{score_def["meter_unit"]}\n'
    abc_content += f'K:{score_def["key"]}\n'
    abc_content += f'L:1/16\n'
    abc_content += f'K: clef={score_def["clef"]}\n'
    for i in range (len(measures_content)):
        measure = measures_content[i]
        for note in measure:
            abc_content += f' {note}'
        abc_content += ' |'
    abc_content += f']'
    return abc_content
abc_content = mei_to_abc(measures_content, score_def)
print(abc_content)

X:1
M:2/4
K:Eb
L:1/16
K: clef=C1
 Z23 | z4 z2 b,2 | b,6 g,2 | e6 d2 | c2 c2 z4 |]
